In [1]:
from src.chat import Chat
from app.models.UserInput import UserInput

In [2]:
chat = Chat()
chat.init_app()

/Users/gcpuppy/Documents/WorkSpace/DataSci/dsde-project/src/database/qdrant.py:37: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv("combined_data.csv")


## Set System Prompt

In [9]:
user_message = "Give me the papers that related to field of aerodynamic especially the military purposes"

user_input = UserInput(
    chat_id="abc",
    message=user_message,
    currentGraph=None
)

# Goal
1. Give "Keyword node"
2. Give "GrapLink from GraphData"

In [14]:
### FIX HERE
system_template_prompt = """You are an AI assistance for encouraging the system to initialize new "Node" from below data that was fetched from QdrantDB. Node have two types which are "Paper" and "Keyword".
Keyword is a short word that related to topic, title or field of the input messages from user while "Paper" is the main content of articles in the database. Your tasks are generating "Keyword" node based on summarizing the content of user messages and "Paper" node, a node that contains more . Both of them follow this structure of node object: 
export interface Node {
title: string
type: 'paper' | 'keyword'
color?: string
citations?: number
year?: number
abstract?: string
authors?: string[]
relevance?: number
}
export interface GraphLink {
source: string
target: string
index?: number
}
Don't forget to ensure that the values that will be assigned are related to data in the databas
This is the guideines for you:
1. Analyze the given user input "Keyword".
2. Create nodes based on related keywords.
3. Give me the response in format of Json of GraphLink
source is the ID of starting node,
target is ID of destination node,
"""
### FIX HERE

chat.set_system_template_prompt(system_template_prompt)

# Set Chat Template Prompt

In [15]:
### FIX HERE
chat_template_prompt = """
{paper_data}
"""
### FIX HERE

chat.set_chat_template_prompt(chat_template_prompt)

## Example of {paper_data}
====================================================================================================================================
ID : 2410.18541_arxiv

Title: On Explaining with Attention Matrices

Year: 2024

Authors: Omar Naim and Nicholas Asher

Source: arxiv

Abstract:   This paper explores the much discussed, possible explanatory link between
attention weights (AW) in transformer models and predicted output. Contrary to
intuition and early research on attention, more recent prior research has
provided formal arguments and empirical evidence that AW are not explanatorily
relevant. We show that the formal arguments are incorrect. We introduce and
effectively compute efficient attention, which isolates the effective
components of attention matrices in tasks and models in which AW play an
explanatory role. We show that efficient attention has a causal role (provides
minimally necessary and sufficient conditions) for predicting model output in
NLP tasks requiring contextual information, and we show, contrary to [7], that
efficient attention matrices are probability distributions and are effectively
calculable. Thus, they should play an important part in the explanation of
attention based model behavior. We offer empirical experiments in support of
our method illustrating various properties of efficient attention with various
metrics on four datasets.

====================================================================================================================================
ID : 2204.13154_arxiv

Title: Attention Mechanism in Neural Networks: Where it Comes and Where it Goes

Year: 2022

Authors: Derya Soydaner

Source: arxiv

Abstract:   A long time ago in the machine learning literature, the idea of incorporating
a mechanism inspired by the human visual system into neural networks was
introduced. This idea is named the attention mechanism, and it has gone through
a long development period. Today, many works have been devoted to this idea in
a variety of tasks. Remarkable performance has recently been demonstrated. The
goal of this paper is to provide an overview from the early work on searching
for ways to implement attention idea with neural networks until the recent
trends. This review emphasizes the important milestones during this progress
regarding different tasks. By this way, this study aims to provide a road map
for researchers to explore the current development and get inspired for novel
approaches beyond the attention.

In [16]:
### See the result of the chat

chat.test_initial_chat(user_input=user_input)

KeyError: '\ntitle'

# Set Detect Additional Data

In [7]:
detect_additional_data_template = """
{user_message}

{current_graph}
"""
chat.set_detect_additional_data_template(detect_additional_data_template)